In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pylab import rcParams
%pylab inline
rcParams['figure.figsize'] = 12, 6
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


In [2]:
d = pd.read_csv('tmp/features_02.csv')
del d['TripType']
del d['Weekday']
del d['Weekday_nan']
dtr = d[d.TripType_0 == 0]
dte = d[d.TripType_0 == 1]
tr = DataFrame({'VisitNumber': dtr.VisitNumber})
trl = DataFrame({'VisitNumber': dtr.VisitNumber})
te = DataFrame({'VisitNumber': dte.VisitNumber})
for c in d.columns:
    if (c.startswith('ScanCount') or
        c.startswith('DepartmentDescription') or
        c.startswith('Weekday')):
        tr[c] = dtr[c]
        te[c] = dte[c]
    if c.startswith('TripType'):
        trl[c] = dtr[c]
del trl['TripType_0']

In [3]:
pred = DataFrame({'VisitNumber': te.VisitNumber})
for c in trl.columns:
    if c.startswith('TripType'):
        print c
        m = XGBClassifier(
            objective='binary:logistic',
            n_estimators=200,
            max_depth=4,
            min_child_weight=4,
            learning_rate=0.03,
            nthread=4)
        m.fit(tr, trl[c])
        p = m.predict_proba(te)[:,1]
        pred[c] = p

TripType_3
TripType_4
TripType_5
TripType_6
TripType_7
TripType_8
TripType_9
TripType_12
TripType_14
TripType_15
TripType_18
TripType_19
TripType_20
TripType_21
TripType_22
TripType_23
TripType_24
TripType_25
TripType_26
TripType_27
TripType_28
TripType_29
TripType_30
TripType_31
TripType_32
TripType_33
TripType_34
TripType_35
TripType_36
TripType_37
TripType_38
TripType_39
TripType_40
TripType_41
TripType_42
TripType_43
TripType_44
TripType_999


In [4]:
pred.to_csv('output/xgb_03_raw.csv', index=False)

In [5]:
def foo(row):
    x = row.max()
    q = row.quantile(0.9)
    row[row < x] = 0
    return row

In [6]:
pred2 = pred.copy()
vn = pred2.VisitNumber
del pred2['VisitNumber']
pred2 = pred2.apply(foo, axis=1)
pred2['VisitNumber'] = vn

In [7]:
pred2.to_csv('output/xgb_03.csv', index=False)